In [0]:
import pyspark
from pyspark.sql import SparkSession

In [0]:
spark=SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression 

In [0]:
training=spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [0]:
lr=LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [0]:
lrModel=lr.fit(training)

In [0]:
lrModel.coefficients

Out[11]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[12]: 0.14228558260358093

In [0]:
training_summary=lrModel.summary

In [0]:
training_summary.rootMeanSquaredError

Out[14]: 10.16309157133015

In [0]:
all_data=spark.read.format('libsvm').load('/FileStore/tables/sample_linear_regression_data.txt')

In [0]:
train_data,test_data=all_data.randomSplit([0.7,0.3])

In [0]:
train_data.describe().show()

+-------+--------------------+
summary| label|
+-------+--------------------+
 count| 361|
 mean|-0.11039756318531466|
 stddev| 10.00669464890077|
 min| -28.046018037776633|
 max| 27.111027963108548|
+-------+--------------------+

In [0]:
test_data.describe().show()

+-------+-------------------+
summary| label|
+-------+-------------------+
 count| 140|
 mean| 1.2039630016399165|
 stddev| 11.061267003585714|
 min|-28.571478869743427|
 max| 27.78383192005107|
+-------+-------------------+

In [0]:
correct_model=lr.fit(train_data)

In [0]:
test_results=correct_model.evaluate(test_data)

In [0]:
test_results.rootMeanSquaredError

Out[23]: 11.296966960980226

In [0]:
unlabeled_data=test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

In [0]:
predictions=correct_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
(10,[0,1,2,3,4,5,...|-2.2111278786375994|
(10,[0,1,2,3,4,5,...|-3.1955381124286655|
(10,[0,1,2,3,4,5,...| 3.0867072139988148|
(10,[0,1,2,3,4,5,...|-0.3891480860289419|
(10,[0,1,2,3,4,5,...| -1.768773097020903|
(10,[0,1,2,3,4,5,...|-0.3185644955708272|
(10,[0,1,2,3,4,5,...|0.18036160894104514|
(10,[0,1,2,3,4,5,...|-3.4361450965605487|
(10,[0,1,2,3,4,5,...| -0.609035714414064|
(10,[0,1,2,3,4,5,...|-1.7115251269416396|
(10,[0,1,2,3,4,5,...| 2.0389139315671327|
(10,[0,1,2,3,4,5,...| 0.933014958867121|
(10,[0,1,2,3,4,5,...|-1.2104521058775943|
(10,[0,1,2,3,4,5,...| 3.6247378153327388|
(10,[0,1,2,3,4,5,...|-0.4918509800097348|
(10,[0,1,2,3,4,5,...|-1.5341484242111003|
(10,[0,1,2,3,4,5,...|-1.0709320123888375|
(10,[0,1,2,3,4,5,...|-1.4153415780985443|
(10,[0,1,2,3,4,5,...|-1.3239509547079402|
(10,[0,1,2,3,4,5,...| 3.4142663825995276|
+--------------------+-------------------+
only showing top 20 rows